In [1]:
import pandas as pd
import json
import numpy as np

In [9]:
import os
os.chdir('/home/airi/PycharmProjects/protagonist_tagger')

In [46]:
for i in [10, 20, 30, 40, 67]:
    data = pd.read_csv('data/resolved.tsv', sep='\t')
    with open('data/orig.txt') as f:
        text = f.read()
    data = data[data.CHARACTER != 'NON-CHARACTER']
    data = data.loc[np.array(data[['START OFFSET', 'END OFFSET']].drop_duplicates().index)]
    data["CHARACTER"].replace({'Right Honourable Lady Catherine de\r\nBourgh': "Right Honourable Lady Catherine de Bourgh",
                         'Mrs.\r\nReynolds': 'Mrs.Reynolds',
                         'Colonel\r\nMiller': 'Colonel Miller',
                         'Miss\r\nGrantley': 'Miss Grantley',
                         'Miss\r\nWebbs': 'Miss Webbs',                     
                        }, inplace=True)

    characters = ['Elizabeth',  'Mr. Darcy',  'Jane',  'Mrs. Bennet',  'Mr. Bingley',  'Mr. Wickham',  'Mr. Collins',  'Lydia',  'Mr. Bennet',  'Right Honourable Lady Catherine de Bourgh',  'Miss Bingley',  'Charlotte Lucas',  'Mr. Gardiner',  'Mrs. Gardiner',  'Kitty',  'Miss Darcy',  'Colonel Fitzwilliam',  'Sir William Lucas',  'Mary',  'Mrs. Hurst',  'Mrs. Phillips',  'Miss de Bourgh',  'Colonel Forster',  'Maria Lucas',  'Mrs.Reynolds',  'Lady Lucas',  'Mrs. Forster',  'Mrs. Long',  'Miss King',  'Mr. Hurst',  'Mr. Denny',  'Hill',  'Mr. Phillip',  'Mrs. Younge',  'Mrs. Jenkinson',  'Mrs. Annesley',  'Lucas',  'Lady Anne Darcy',  'Nicholls',  'Mr. Jones',  'Pratt',  'Captain Carter',  'Lady Metcalf',  'Miss Pope',  'Sir Lewis de Bourgh',  'William Goulding',  'Pen',  'Chamberlayne',  'Gouldings',  'Mr. Robinson',  'Harriet',  'Richard',  'Haggerston',  'General',  'Colonel Miller',  'Sarah',  'Mr. Morris',  'Sally',  'Mr. Stone',  'Miss Watson',  'Miss Grantley',  'Archbishop',  'Miss Webbs',  'Clarke',  'Lord',  'Dawson',  'John'][:i]
    data = data[data.CHARACTER.isin(characters)]
    
    results = {'content': text, 'entities': []}
    for r in data[data.CHARACTER.isin(characters)].iterrows():
        results['entities'].append([r[1]["START OFFSET"], r[1]["END OFFSET"], r[1]["CHARACTER"]])

    for r in data[~data.CHARACTER.isin(characters)].iterrows():
        results['entities'].append([r[1]["START OFFSET"], r[1]["END OFFSET"], "PERSON"])
        
    if not os.path.exists('data/testing_sets/pride_and_prejudice_' + str(i)):
        os.makedirs('data/testing_sets/pride_and_prejudice_' + str(i))
    if not os.path.exists('data/lists_of_characters/pride_and_prejudice_' + str(i)):
        os.makedirs('data/lists_of_characters/pride_and_prejudice_' + str(i))
        
    with open('data/testing_sets/pride_and_prejudice_' + str(i) + '/pride_and_prejudice.json', 'w') as f:
        f.write(json.dumps([results]))
        
    with open('data/lists_of_characters/pride_and_prejudice_' + str(i) + '/pride_and_prejudice', 'w') as f:
        f.write('\n'.join(characters))

In [62]:
import pandas as pd
import json
import numpy as np
import os
from collections import Counter

protagonist_data_dir = 'experiments/final_protagonist/'
title = 'pride_and_prejudice'
coreference_data_dir = 'experiments/final_coref/'

with open(os.path.join(protagonist_data_dir, title + '.json')) as f:
    protagonist_results = json.loads(f.read())

with open(os.path.join(protagonist_data_dir, 'ner', title + '.json')) as f:
    ner_results = json.loads(f.read())

with open(os.path.join(coreference_data_dir, title + '.json')) as f:
    coref_results = json.loads(f.read())[0]
coref_results['content'] = coref_results['content']

protagonist_merged = {'content': '\n\n'.join([x['content'] for x in protagonist_results])}
entities = []
delay = [0] + list(np.cumsum([len(x['content'])+2 for x in protagonist_results]))
for i in range(len(protagonist_results)):
    for ent in protagonist_results[i]['entities']:
        entities.append([ent[0] + delay[i], ent[1] + delay[i], ent[2]])
protagonist_merged['entities'] = entities

ner_merged = {'content': '\n\n'.join([x['content'] for x in ner_results])}
entities = []
delay = [0] + list(np.cumsum([len(x['content'])+2 for x in ner_results]))
for i in range(len(ner_results)):
    for ent in ner_results[i]['entities']:
        entities.append([ent[0] + delay[i], ent[1] + delay[i], ent[2], ent[3]])
ner_merged['entities'] = entities

skip_ambiguous = False
majority_vote = True
random_vote = False
combined_results = []
matched_clusters = 0
matched_entities = [False] * len(ner_merged['entities'])
for cluster in coref_results['clusters']:
    possible_matches = []
    matches = []
    for ent_id, (ent_ner, ent_protagonist) in enumerate(zip(ner_merged['entities'], protagonist_merged['entities'])):
        ent_personal = None
        if ent_ner[3]:
            ent_personal = [ent_protagonist[0] - len(ent_ner[3]) - 1, ent_protagonist[1], ent_protagonist[2]]
        if ent_protagonist[:2] in cluster or (ent_personal and ent_personal[:2] in cluster):
            possible_matches.append(ent_protagonist[2])
            matched_entities[ent_id] = True
    match = None
    if len(possible_matches) > 1:
        if skip_ambiguous:
            continue
        elif random_vote:
            match = np.random.choice(possible_matches)
        elif majority_vote:
            counts = Counter(possible_matches).most_common()
            matches = [counts[0][0]]
            max_count = counts[0][1]
            i = 0
            while i<len(counts) and counts[i][1] == max_count:
                matches.append(counts[i][0])
                i += 1
            match = np.random.choice(matches)

    elif len(possible_matches) == 1:
        match = possible_matches[0]
        
    if match:
        matched_clusters += 1
        for mention in cluster:
            combined_results.append((mention[0], mention[1], match))


print("Number of NER entities", len(protagonist_merged['entities']))
print("Number of NER entities", len(ner_merged['entities']))
print("Number of PERSON entities", len([e for e in protagonist_merged['entities'] if e[2] == "PERSON"]))

print("Number of clusters", len(coref_results['clusters']))
print("Number of mentions", sum([len(c) for c in coref_results['clusters']]))
print("Number of matched clusters", matched_clusters)
print("Number of final mentions", len(combined_results))

for ent_id, ent in enumerate(protagonist_merged['entities']):
    if not matched_entities[ent_id] and ent not in combined_results:
        ent[0] = int(ent[0])
        ent[1] = int(ent[1])
        combined_results.append(tuple(ent))

print("Number of final mentions", len(combined_results))

        
results_dict = [{'content': ner_merged['content'], 'entities': combined_results }]
path = 'experiments/final_combined/pride_and_prejudice.json'

if not os.path.exists(os.path.dirname(path)):
    os.makedirs(os.path.dirname(path))

with open(path, 'w+') as result:
    json.dump(results_dict, result)

Number of NER entities 3600
Number of NER entities 3600
Number of PERSON entities 844
Number of clusters 1377
Number of mentions 23065
Number of matched clusters 133
Number of final mentions 15618
Number of final mentions 16663


In [63]:
1

1

In [15]:
results["content"][378:778]

'"My dear Mr. Bennet," said his lady to him one day, "have you heard that\nNetherfield Park is let at last?"\n\nMr. Bennet replied that he had not.\n\n"But it is," returned she; "for Mrs. Long has just been here, and she\ntold me all about it."\n\nMr. Bennet made no answer.\n\n"Do you not want to know who has taken it?" cried his wife impatiently.\n\n"You want to tell me, and I have no objection to hearing it.'

In [16]:
data[data['START OFFSET'] <= 778]

,SPAN,CHARACTER,START OFFSET,END OFFSET,CHAPTER #,PASSAGE #
8,Mr. Bennet,Mr. Bennet,387,397,1,0
9,lady,Mrs. Bennet,409,413,1,0
10,him,Mr. Bennet,417,420,1,0
11,you,Mr. Bennet,436,439,1,0
12,Mr. Bennet,Mr. Bennet,486,496,1,0
13,he,Mr. Bennet,510,512,1,0
14,she,Mrs. Bennet,545,548,1,0
15,Mrs. Long,Mrs. Long,555,564,1,0
16,she,Mrs. Long,589,592,1,0
17,me,Mrs. Long,598,600,1,0


In [17]:
[(c, results["content"][c[0]:c[1]])  for c in combined_results if c[0] <= 778]

[((379, 397, 'Mr. Bennet'), 'My dear Mr. Bennet'),
 ((405, 408, 'Mr. Bennet'), 'his'),
 ((417, 420, 'Mr. Bennet'), 'him'),
 ((436, 439, 'Mr. Bennet'), 'you'),
 ((486, 496, 'Mr. Bennet'), 'Mr. Bennet'),
 ((510, 512, 'Mr. Bennet'), 'he'),
 ((617, 627, 'Mr. Bennet'), 'Mr. Bennet'),
 ((649, 652, 'Mr. Bennet'), 'you'),
 ((695, 698, 'Mr. Bennet'), 'his'),
 ((736, 738, 'Mr. Bennet'), 'me'),
 ((528, 530, 'PERSON'), 'it'),
 ((684, 686, 'PERSON'), 'it'),
 ((555, 564, 'Mrs. Long'), 'Mrs. Long'),
 ((589, 592, 'Mrs. Long'), 'she'),
 ((391, 397, 'Mr. Bennet'), 'Bennet')]

In [33]:
[(c, results["content"][c[0]:c[1]])  for c in coref_results['clusters'][5]]

[([528, 530], 'it'),
 ([684, 686], 'it'),
 ([860, 871], 'Netherfield'),
 ([996, 1005], 'the place'),
 ([1038, 1040], 'it'),
 ([1173, 1182], 'the house'),
 ([22242, 22259], 'Netherfield House'),
 ([22276, 22278], 'it'),
 ([22289, 22291], 'it'),
 ([22404, 22407], 'its'),
 ([22425, 22427], 'it'),
 ([270210, 270223], 'the Parsonage'),
 ([270261, 270263], 'it'),
 ([270427, 270440], 'the Parsonage'),
 ([270490, 270514], 'the house standing in it'),
 ([270722, 270731], 'the house'),
 ([309456, 309465], 'the house'),
 ([309562, 309564], 'it'),
 ([309573, 309575], 'it'),
 ([309662, 309664], 'it')]

In [40]:
protagonist_merged['entities']

[[391, 397, 'Mr. Bennet'],
 [490, 496, 'Mr. Bennet'],
 [560, 564, 'Mrs. Long'],
 [621, 627, 'Mr. Bennet'],
 [845, 849, 'Mrs. Long'],
 [860, 871, 'PERSON'],
 [1066, 1072, 'Mr. Morris'],
 [1123, 1133, 'PERSON'],
 [1232, 1239, 'Mr. Bingley'],
 [1449, 1455, 'Mr. Bennet'],
 [1944, 1951, 'Mr. Bingley'],
 [2318, 2325, 'Mr. Bingley'],
 [2509, 2516, 'Sir William Lucas'],
 [2526, 2531, 'Lady Lucas'],
 [2761, 2768, 'Mr. Bingley'],
 [2969, 2974, 'Elizabeth'],
 [3015, 3020, 'Elizabeth'],
 [3103, 3107, 'Jane'],
 [3138, 3143, 'Lydia'],
 [3309, 3314, 'Elizabeth'],
 [3372, 3378, 'Mr. Bennet'],
 [4010, 4016, 'Mr. Bennet'],
 [4480, 4486, 'Mr. Bennet'],
 [4537, 4544, 'Mr. Bingley'],
 [4874, 4881, 'Mr. Bingley'],
 [4896, 4901, 'Elizabeth'],
 [4943, 4950, 'Mr. Bingley'],
 [5048, 5057, 'Elizabeth'],
 [5116, 5120, 'Mrs. Long'],
 [5173, 5177, 'Mrs. Long'],
 [5330, 5336, 'Mr. Bennet'],
 [5411, 5417, 'Mrs. Bennet'],
 [5544, 5549, 'Kitty'],
 [5637, 5642, 'Kitty'],
 [5764, 5769, 'Kitty'],
 [5812, 5817, 'Elizabeth'